In [1]:
class Node:
    def __init__(self, name):
        self.name = name 


class directory(Node):
    def __init__(self,name):
        super().__init__(name)
        self.children = {}  # {} # can have directories and files
    
    
class file(Node):
    def __init__(self,name,content = ""):
        super().__init__(name)
        self.content = content  # {}
    
    

class FileSystem():
    def __init__(self):
        self.root = directory("")
    
    
    def mkdir(self,path):
        dir_names = [i for i in path.split("/") if i]
        node = self.root 
        
        for each in dir_names:
            if each not in node.children:
                node.children[each] = directory(each)
            node = node.children[each]
        
        return 

    def addContentToFile(self,path,content):
        names = [i for i in path.split("/") if i]
        dir_names = names[:-1]
        file_name = names[-1]
        node = self.root
        for each in dir_names:
            if each not in node.children:
                raise Exception("Directory {each} does not Exist for the following Directories: {dir_names}")
            node = node.children[each]
        File = file(file_name,content)
        node.children[file_name] = File
        return 
    
    def ls(self,path):
        names = [i for i in path.split("/") if i]
        node = self.root
        for each in names:
            if each not in node.children:
                raise Exception("Directory {each} does not Exist for the following Directories: {names}")
            node = node.children[each]
        
        if isinstance(node,file):
            return node.name 

        return sorted(node.children.keys())
        
        
        

In [2]:
# file system
fs = FileSystem()

fs.ls("/") 
# Returns []

fs.mkdir("/a/b/c")
fs.addContentToFile("/a/b/c/d", "hello")

fs.ls("/") 
# Returns ["a"]

fs.ls("/a/b/c")
# Returns ["d"] -> "d" is the file inside /a/b/c

fs.readContentFromFile("/a/b/c/d")
# Returns "hello"

AttributeError: 'FileSystem' object has no attribute 'readContentFromFile'

Problem Statement:We need to design a Bigram Frequency Analyzer.You will receive a stream of sentences. Your system must parse these sentences and track how often pairs of consecutive words (bigrams) appear.Core Requirements:Ingest data: Process sentences one by one.Query data: Return the frequency of a specific bigram (pair of words).Example:Input: "The quick brown fox jumps. The quick brown fox."Bigrams for first sentence: (the, quick), (quick, brown), (brown, fox), (fox, jumps)Query ("quick", "brown") $\to$ Returns 2.Query ("fox", "jumps") $\to$ Returns 1.Constraints:Case-insensitive ("The" == "the").Ignore punctuation ("fox." == "fox").Handle whitespace properly.

In [ ]:
from collections import defaultdict
class BigRam():
    def __init__(self):
        self.counts = defaultdict(int)
        # self.counts = defaultdict(lambda : defaultdict(int))
    
    
    def ingest(self,data):
        word = data.lower().strip().split()
        for i in range(1,len(word)):
            self.counts[(word[i-1],word[i])] += 1
            # self.counts[word[i-1]][word[i]] += 1
        return
    
    def query(self,word1,word2):
        return self.counts[(word1,word2)]

    
    def get_top_k_following(self, word: str, k: int) -> List[str]:
        res = []
        
        for chr,count in self.counts[word].items():
            heapq.heappush(res, (count,chr))
            
            if len(res) > k:
                heapq.heappop(res)
        
        return [char for _,char in res]